In [1]:
#crawl_iwencai_flags_history
from opentrader.core.crawler import *
from opentrader.agents.xueqiu.api import *
from opentrader.agents.ths.api import *
from opentrader.core.ticker import *
from opentrader.common.db import db_ot
from opentrader.common.utils import d2dt
from pymongo.errors import *

start_date = gen_time('2005-01-01 00:00:00').date()
end_date = gen_time('2015-12-17 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)
print(len(dates))

2660


In [2]:
# now use dates to fetch history flags data and store into mongodb
for date in dates:
    print(date, end="")
    f = open('iwencai_flags.json', 'r')
    try:
        flags = json.loads(f.read())
    except:
        flags = []
    flags_lock = threading.RLock()
    
    class FlagCrawler(threading.Thread):
        def __init__(self):
            threading.Thread.__init__(self)
            self.ths = THSAPI()

        def run(self):
            sys.stdout.write('b')
            sys.stdout.flush()
            i = 0
            while True:
                flags_lock.acquire()
                try:
                    flag = flags.pop()
                except IndexError:
                    flags_lock.release()
                    break
                flags_lock.release()
                
                # check whether already exists at the database
                if (db_ot.iwencai_flag_history.find({'flag':flag, 'date':d2dt(date)}).count() > 0):
                    continue
                # search each flag at iwencai
                try:
                    result = self.ths.query_iwencai(date.strftime('%Y年%m月%d日')+flag)
                except Exception as e:
                    sys.stdout.write(flag+str(e))
                    continue
                try:
                    result = result['result']
                except TypeError as e:
                    sys.stdout.write(result)
                    sys.stdout.write(str(e))
                    continue
                i += 1
                if i%2 == 0:
                    sys.stdout.write(".")
                    sys.stdout.flush()
                # insert into db
                try:
                    db_ot.iwencai_flag_history.insert_one({'flag':flag, 'date':d2dt(date),'symbols':[each[0] for each in result]})
                except DuplicateKeyError as e:
                    sys.stdout.write(str(e))
            sys.stdout.write('e%d' % (i))
            sys.stdout.flush()
    
    threads = []
    for i in range(20):
        thread = FlagCrawler()
        threads.append(thread)
        thread.start()

    for each in threads:
        each.join()
    print("")

2005-01-04bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-05bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-06bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-07bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-10bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-11bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-12bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-13bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-14bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-17bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-18bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-19bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-20bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-01-21bbbbbbbbbbbbbbbbbbbbe0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0e0
2005-0